In [7]:
import requests
import os
import json
import time
import tempfile
import GEA_functions as GEA
from GEA_classified import API_KEY as api_key


#BEFORE YOU CLICK 'RUN ALL'
# you can collapse function definitions so its more readable
# 

In [2]:
#DEFAULT VARIABLES 
#do not change unless you know what you are doing
#if you do please make clear what you change and why

mode = "competitive" #no caps

allMaps = {"/Game/Maps/Ascent/Ascent": "Ascent", 
"/Game/Maps/Bonsai/Bonsai": "Split",
"/Game/Maps/Canyon/Canyon": "Fracture",
"/Game/Maps/Duality/Duality": "Bind",
"/Game/Maps/Foxtrot/Foxtrot": "Breeze",
"/Game/Maps/Jam/Jam": "Lotus",
"/Game/Maps/Juliett/Juliett": "Sunset",
"/Game/Maps/Pitt/Pitt": "Pearl",
"/Game/Maps/Port/Port": "Icebox",
"/Game/Maps/Triad/Triad": "Haven",
"/Game/Maps/Infinity/Infinity": "Abyss",
"/Game/Maps/HURM/HURM_Alley/HURM_Alley": "District",
"/Game/Maps/HURM/HURM_Bowl/HURM_Bowl": "Kasbah",
"/Game/Maps/HURM/HURM_Helix/HURM_Helix": "Drift",
"/Game/Maps/HURM/HURM_Yard/HURM_Yard": "Piazza",
"/Game/Maps/Poveglia/Range": "The Range"
}

allGuns = {
    "": "No Weapon",
    "63E6C2B6-4A8E-869C-3D4C-E38355226584": "Odin",
    "55D8A0F4-4274-CA67-FE2C-06AB45EFDF58": "Ares",
    "9C82E19D-4575-0200-1A81-3EACF00CF872": "Vandal",
    "AE3DE142-4D85-2547-DD26-4E90BED35CF7": "Bulldog",
    "EE8E8D15-496B-07AC-E5F6-8FAE5D4C7B1A": "Phantom",
    "EC845BF4-4F79-DDDA-A3DA-0DB3774B2794": "Judge",
    "910BE174-449B-C412-AB22-D0873436B21B": "Bucky",
    "44D4E95C-4157-0037-81B2-17841BF2E8E3": "Frenzy",
    "29A0CFAB-485B-F5D5-779A-B59F85E204A8": "Classic",
    "1BAA85B4-4C70-1284-64BB-6481DFC3BB4E": "Ghost",
    "E336C6B8-418D-9340-D77F-7A9E4CFE0702": "Sheriff",
    "42DA8CCC-40D5-AFFC-BEEC-15AA47B42EDA": "Shorty",
    "A03B24D3-4319-996D-0F8C-94BBFBA1DFC7": "Operator",
    "4ADE7FAA-4CF1-8376-95EF-39884480959B": "Guardian",
    "5F0AAF7A-4289-3998-D5FF-EB9A5CF7EF5C": "Outlaw",
    "C4883E50-4494-202C-3EC3-6B8A9284F00B": "Marshal",
    "462080D1-4035-2937-7C09-27AA2A5C27A7": "Spectre",
    "F7E1B454-4AD4-1063-EC0A-159E56B58941": "Stinger",
    "2F59173C-4BED-B6C3-2191-DEA9B58BE9C7": "Melee"
}
allArmor = {
    #ArmorID conversions
    "": "No Shield",
    "822BCAB2-40A2-324E-C137-E09195AD7692": "Heavy Shields",
    "4DEC83D5-4902-9AB3-BED6-A7A390761157": "Light Shields"
}


#FOR ENTRIESE THAT ARE ALL ZERO, maybe make the multipliers 1 instead of 0
mapCoords = {
    "Ascent": {
        "xMultiplier": 7e-05,
        "yMultiplier": -7e-05,
        "xScalar": 0.813895,
        "yScalar": 0.573242
    },
    "Split": {
        "xMultiplier": 7.8e-05,
        "yMultiplier": -7.8e-05,
        "xScalar": 0.842188,
        "yScalar": 0.697578
    },
    "Fracture": {
        "xMultiplier": 7.8e-05,
        "yMultiplier": -7.8e-05,
        "xScalar": 0.556952,
        "yScalar": 1.155886
    },
    "Bind": {
        "xMultiplier": 5.9e-05,
        "yMultiplier": -5.9e-05,
        "xScalar": 0.576941,
        "yScalar": 0.967566
    },
    "Breeze": {
        "xMultiplier": 7e-05,
        "yMultiplier": -7e-05,
        "xScalar": 0.465123,
        "yScalar": 0.833078
    },
    "District": {
        "xMultiplier": 0,
        "yMultiplier": 0,
        "xScalar": 0,
        "yScalar": 0
    },
    "Kasbah": {
        "xMultiplier": 0,
        "yMultiplier": 0,
        "xScalar": 0,
        "yScalar": 0
    },
    "Drift": {
        "xMultiplier": 0,
        "yMultiplier": 0,
        "xScalar": 0,
        "yScalar": 0
    },
    "Piazza": {
        "xMultiplier": 0,
        "yMultiplier": 0,
        "xScalar": 0,
        "yScalar": 0
    },
    "Abyss": {
        "xMultiplier": 8.1e-05,
        "yMultiplier": -8.1e-05,
        "xScalar": 0.5,
        "yScalar": 0.5
    },
    "Lotus": {
        "xMultiplier": 7.2e-05,
        "yMultiplier": -7.2e-05,
        "xScalar": 0.454789,
        "yScalar": 0.917752
    },
    "Sunset": {
        "xMultiplier": 7.8e-05,
        "yMultiplier": -7.8e-05,
        "xScalar": 0.5,
        "yScalar": 0.515625
    },
    "Pearl": {
        "xMultiplier": 7.8e-05,
        "yMultiplier": -7.8e-05,
        "xScalar": 0.480469,
        "yScalar": 0.916016
    },
    "Icebox": {
        "xMultiplier": 7.2e-05,
        "yMultiplier": -7.2e-05,
        "xScalar": 0.460214,
        "yScalar": 0.304687
    },
    "The Range": {
        "xMultiplier": 0,
        "yMultiplier": 0,
        "xScalar": 0,
        "yScalar": 0
    },
    "Haven": {
        "xMultiplier": 7.5e-05,
        "yMultiplier": -7.5e-05,
        "xScalar": 1.09345,
        "yScalar": 0.642728
    }
}

DBUG = False #set to true to unleash the debug statements
if DBUG:
    dbug_print = print
else:
    dbug_print = lambda *args, **kwargs: None



#USELESS but i am keeping for future reference i will prob delete it anyway
# from collections import deque

# all_requests = deque()

# def check_time():
    
#     global all_requests

#     print(f"\t\t\trequests made in the last min: [{len(all_requests)}]")
#     if(len(all_requests) >= 30):
#         timediff = time.time() - all_requests.popleft()
#         print(f"\t\t\t\ttime elapsed from oldest request: [{timediff}]")
#         if(timediff < 60):
#             print(f"\t\t\t\tsleeping for [{60-timediff}] sec")
#             time.sleep(60-timediff)
        
#         check_time()
# def put_time():
#     global all_requests

#     print("\t\t\trequest made.")
#     all_requests.append(time.time())


#ensures that we dont exceed the max requests. write this after every API request!!!!!
def limit_check(response):
    print("\t\t\trequest made.")
    print(f"\t\t\t\tAlleged requests made: [{response.headers['x-ratelimit-limit']}]/30")
    if(int(response.headers['x-ratelimit-limit']) >= 30):
        print(f"\t\t\t\tsleeping for [{response.headers['x-ratelimit-reset']} + 2] sec")
        time.sleep(int(response.headers['x-ratelimit-reset']) + 2)     

#ALL API REQUESTS SHOULD LOOK LIKE THIS
#       print("Making request...")
#       response = <API request statmement>
#       limit_check(response)


In [3]:
#FUNCTION DEFINITIONS(you can collapse)
#These all directly access API which is why theyre here

#Accesses API
#returns puuid of player
def get_puuid(name: str, tag: str, api_key: str):

    url = f"https://api.henrikdev.xyz/valorant/v1/account/{name}/{tag}"
    headers = {'Authorization': f'{api_key}'}

    print("Making request...")
    response = requests.get(url, headers=headers)
    limit_check(response)



    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch puuid: {response.status_code}")
        error = response.json()['errors'][0]['message']
        details= response.json()['errors'][0]['details']
        code = response.json()['status']
        if details == "null":
            details = ""     
        else:
            details = f"Details: {details}"
        response_message = f"{error} Code:{code}....{details}"
        print(response_message, "\nEXITING PROGRAM")
        quit()

#Accesses API
#returns the player's raw data match history
#format is here: https://valapidocs.techchrism.me/endpoint/match-history 
def get_raw(puuid: str,api_key: str,queue: str, startIndex: int, endIndex: int):
    url = f"https://api.henrikdev.xyz/valorant/v1/raw"
    headers = {
        'Authorization': f'{api_key}',
    }
    body = {
        "type": "matchhistory",
        "value": puuid,
        "region": "na",
        "queries": f"?startIndex={startIndex}&endIndex={endIndex}&queue={queue}"
    }
    
    print("Making request...")
    response = requests.post(url, headers=headers, json=body)
    limit_check(response)

 
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch raw_data: {response.status_code}")
        error = response.json()['errors'][0]['message']
        details= response.json()['errors'][0]['details']
        code = response.json()['status']
        if details == "null":
            details = ""     
        else:
            details = f"Details: {details}"
        response_message = f"{error} Code:{code}....{details}"
        print(response_message, "\nEXITING PROGRAM")
        quit()

#Accesses API
#returns match details of matchID
#format is here: https://valapidocs.techchrism.me/endpoint/match-details 
def get_match_details(matchID: str,api_key: str):
    url = f"https://api.henrikdev.xyz/valorant/v1/raw"
    headers = {
        'Authorization': f'{api_key}',
    }
    body = {
        "type": "matchdetails",
        "value": matchID,
        "region": "na",
    }
    
    
    print("Making request...")
    response = requests.post(url, headers=headers, json=body)
    limit_check(response)
 
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch raw_data: {response.status_code}")
        error = response.json()['errors'][0]['message']
        details= response.json()['errors'][0]['details']
        code = response.json()['status']
        if details == "null":
            details = ""     
        else:
            details = f"Details: {details}"
        response_message = f"{error} Code:{code}....{details}"
        print(response_message, "\nEXITING PROGRAM")
        quit()


In [4]:
#READ THIS                                   
#FUNCTION DEFITIONS (you can collapse all of these)
#these access and save data

import statistics

#Does not access API
#auxillary function for the get_data functions
#returns the new team configuration(flips teams)
#team_side is either string or dict, depending on if we are analyzing individual player or all player
def switch_teams(team_side):
    dbug_print("---SWITCHING TEAMS---")
    if isinstance(team_side, dict):

        for key, value in team_side.items(): #allplayer analysis
            if value['Side'] == "Attack":
                team_side[key]['Side'] = "Defense"
            elif value['Side'] == "Defense":
                team_side[key]['Side'] = "Attack"

    elif isinstance(team_side, str): #player analysis

        if team_side == "Attack":
            team_side = "Defense"
        else:
            team_side = "Attack"

    else:
        print("team_side has type ", type(team_side))
        raise ValueError("Invalid team side")

#Does not access API
#auxillary function for analyze_match
#mutates output_dict, does it for specific player
def get_data_for_player(curr_match: dict, map_name: str, puuid: str, output_dict: dict):
    
    player_team = next((player['teamId'] for player in curr_match['players'] if player['subject'] == puuid), None)
    if player_team == "Red": #if player_team is None, player was not found
        team_side = "Attack"
    elif player_team == "Blue":
        team_side = "Defense"



    round_count = 0
    kill_array_index = 0
    for round_result in curr_match['roundResults']:
        if kill_array_index < len(curr_match['kills']): #as long as there are still kills to be checked, we continue
            dbug_print("\n")
            
            round_count += 1
            dbug_print('currently examining new round [', round_count, "]")
            dbug_print('  player is on ', team_side)

            if round_count > 24 or ( round_count % 12 == 1 and round_count > 1):  #switch teams when needed(after round 12,and in OT)  *not explicitly tested to be correct
               
                switch_teams( team_side)


                
        
            #get specified player's stats
            for player_stat in round_result['playerStats']:
                if player_stat['subject'] == puuid:
                    dbug_print("\tplayer stats found")
                    curr_round_stats = player_stat
                    break
            
            #location data will need these things
            loadout_value = curr_round_stats['economy']['loadoutValue']
            weapon = allGuns[curr_round_stats['economy']['weapon']]
            armor  = allArmor[curr_round_stats['economy']['armor']]



            #determine where to put the next entry in the array by loadout value
            array_spot = (int)(loadout_value/50)
            dbug_print("\tloadout value[", loadout_value, "]->array spot[", array_spot, "]")
        
        
        #calculate round length

            #calculate start time 
            round_start_time = curr_match['kills'][kill_array_index]['gameTime'] - curr_match['kills'][kill_array_index]['roundTime']
            
            #calculate start time of next round, if there is no next round, time = -1
            next_round_start_time = -1
            for kill_info in curr_match['kills']:
                if kill_info['round'] == round_count + 1:
                    next_round_start_time = kill_info['gameTime'] - kill_info['roundTime']
                    break
        

            #subtract the buy phase time
            buy_phase = 0
            if(round_count == 1 or round_count == 13 or round_count == 25):
                buy_phase = 45000
            else:
                buy_phase = 30000

            if(next_round_start_time == -1):
                round_length = curr_match['matchInfo']['gameLengthMillis'] - round_start_time #might be wrong because gameLengthMillis might not start from the start of round 1
            else:
                round_length = next_round_start_time - round_start_time - buy_phase
        #round length calculated
        
            dbug_print('\t\tround start(ms)  = ', round_start_time)
            dbug_print('\t\tnext round start(ms)  = ', next_round_start_time)
            dbug_print('\t\tbuy phase(ms)    =', buy_phase)
            dbug_print("\t\tround length(ms) = ", round_length)



            
            #loop through all kills in the current round
            while(kill_array_index < len(curr_match['kills']) and curr_match['kills'][kill_array_index]['round']+1 == round_count):
                dbug_print('\t\t\tcurrently examining kills on round ', curr_match['kills'][kill_array_index]['round']+1, 'should match ', round_count)
                curr_kill_info = curr_match['kills'][kill_array_index]

                if(curr_kill_info['victim'] == puuid):#our player died!

                    death_time_percentage = (int)(round(curr_kill_info['roundTime']/round_length, 2)*100)
                    dbug_print("\t\t\t\tdeath_time[0,100]: ", death_time_percentage, "%")#DBUG

                    #GET RID OF STUPID ISO ULT DATA
                    if abs(curr_kill_info['victimLocation']['x']) <= 20000 and abs(curr_kill_info['victimLocation']['y']) <= 20000:
                        
                        dbug_print('\t\t\t\tINSERTING death INTO SLOT: [', array_spot, "]")
                        output_dict[map_name][team_side]['death_info'][array_spot].append({

                            'Location': {'x': curr_kill_info['victimLocation']['x'], 'y':curr_kill_info['victimLocation']['y']},
                            'Armor' : armor,
                            
                        })

                    #add to the death time
                    output_dict[map_name]['All_Death_Times'][death_time_percentage]+=1
                
                if(curr_kill_info['killer'] == puuid):#our player got a kill!
                    
                    kill_time_percentage = (int)(round(curr_kill_info['roundTime']/round_length,2)*100)
                    dbug_print("\t\t\t\tkill_time[0,100]: ", kill_time_percentage, "%")#DBUG

                    #add kill info (a bit more complicated)
                    for player_loc in curr_kill_info['playerLocations']:

                        if player_loc['subject'] == puuid:

                            #GET RID OF STUPID ISO ULT DATA
                            if abs(player_loc['location']['x']) <= 20000 and abs(player_loc['location']['y']) <= 20000:
                                
                                dbug_print('\t\t\t\tINSERTING kill INTO SLOT: [', array_spot, "]")
                                
                                #add datapoint to output
                                output_dict[map_name][team_side]['kill_info'][array_spot].append({
                                    'Location': {'x': player_loc['location']['x'], 'y': player_loc['location']['y']},
                                    'ViewRadians': player_loc['viewRadians'],
                                    'Weapon': weapon
                                })
                            break
                            

                    #add to the kill time
                    output_dict[map_name]['All_Kill_Times'][kill_time_percentage]+=1
                


                kill_array_index += 1

# UNFINISHSED
#Does not access API 
#auxillary function for analyze_match
#mutates output_dict, different algorithm for all players
def get_data_for_all_players(curr_match: dict, map_name: str, output_dict: dict):
    

    all_players_stats = {player['subject']: {"Side": "Attack" if player['teamId'] == "Red" else "Defense" if player['teamId'] == "Blue" else "MISSING_DATA", "Loadout_Value": None, "Weapon": None, "Armor": None} for player in curr_match['players']}
    round_count = 0
    for round_result in curr_match['roundResults']:
        round_count += 1

        dbug_print('\tcurrently examining new round [', round_count, "]")

        if round_count > 24 or ( round_count % 12 == 1 and round_count > 1):  #switch teams when needed(after round 12,and in OT)  *not explicitly tested to be correct
            switch_teams( all_players_stats)

        #get all the players buy values, weapons, armor (this may not be very effective, because we are getting all this info for players even if they didnt get a kill this round)
        dbug_print("\t\tgetting all player economies")
        if round_result['playerEconomies'] is not None and isinstance(round_result['playerEconomies'], list):
            for curr_player_economy in round_result['playerEconomies']:
                all_players_stats[curr_player_economy['subject']]['Loadout_Value'] = curr_player_economy['loadoutValue']
                all_players_stats[curr_player_economy['subject']]['Weapon'] = allGuns[curr_player_economy['weapon']]
                all_players_stats[curr_player_economy['subject']]['Armor'] = allArmor[curr_player_economy['armor']]
        else:
            break

        
        #time to pull out data
        if round_result['playerStats'] is not None and isinstance(round_result['playerStats'], list):
            for player_stat in round_result['playerStats']:

                if player_stat['kills'] is not None and isinstance(player_stat['kills'], list):
                    for curr_kill in player_stat['kills']:
                        dbug_print("\t\t\tProcessing new kill/death in round [", round_count, "]")
                        #put in DEATH INFO
                        
                        #GET RID OF STUPID ISO ULT DATA
                        if abs(curr_kill['victimLocation']['x']) > 20000 or abs(curr_kill['victimLocation']['y']) > 20000:
                            continue

                        #calculate where we put datapoint
                        array_spot = (int)(all_players_stats[curr_kill['victim']]['Loadout_Value']/50)
                    
                        dbug_print('\t\t\t\tINSERTING death INTO SLOT: [', array_spot, "]")

                        #add datapoint to output
                        output_dict[map_name][all_players_stats[curr_kill['victim']]['Side']]['death_info'][array_spot].append( {
                        
                            'Location': {'x': curr_kill['victimLocation']['x'], 'y':curr_kill['victimLocation']['y']},
                            'Armor' : all_players_stats[curr_kill['victim']]['Armor']

                        })

                        #put in KILL INFO
                        array_spot = (int)(all_players_stats[curr_kill['killer']]['Loadout_Value']/50)
                    
                        dbug_print('\t\t\t\tINSERTING kill INTO SLOT: [', array_spot, "]")

                        #in case killer location is not found, we skip this(killer location not found is actually possible to happen!)
                        for item in curr_kill['playerLocations']:
                            if item['subject'] == curr_kill['killer']:
                                killer_location = item
                                break
                        else:
                            continue

                        #GET RID OF STUPID ISO ULT DATA
                        if abs(killer_location['location']['x']) > 20000 or abs(killer_location['location']['y']) > 20000:
                            continue

                        output_dict[map_name][all_players_stats[curr_kill['killer']]['Side']]['kill_info'][array_spot].append({
                        
                            'Location': {'x': killer_location['location']['x'], 'y':killer_location['location']['y']},
                            'ViewRadians' : killer_location['viewRadians'],
                            'Weapon' : all_players_stats[curr_kill['killer']]['Weapon']

                        })
                else:
                    continue         
        else:
            break
        

#Does not access API
#auxillary function for get_all_location_data
#given match details (format: https://valapidocs.techchrism.me/endpoint/match-details)
#mutates output_dict by placing new data inside
def analyze_match(curr_match: dict, puuid: str, output_dict: dict):
    

    map_id = curr_match['matchInfo']['mapId']
    map_name = allMaps[map_id]

    analyze_all_players = puuid == ""

    print(f"----CURRENT MAP: [{map_name}]----")

    if map_name not in output_dict:
        if(analyze_all_players):
            output_dict[map_name] = {
                
                'num_matches': 0, #number of matches analyzed for this specific map

               #No kill/death times for allplayers
                "Attack": {
                    #make these all linkedlists which apparently is using deque()
                    "kill_info": [[] for _ in range(160)],
                    "death_info": [[] for _ in range(160)]
                    #WE ARE DITCHING THE PLANT INFO!, making a separate dictionary that gets ALL plant data and defuse data(not just specific player)
                },
                "Defense": {
                    "kill_info": [[] for _ in range(160)],
                    "death_info": [[] for _ in range(160)]
                }
            }
        else:
            output_dict[map_name] = {

                'num_matches': 0, #number of matches analyzed for this specific map

                'All_Kill_Times' : [0]*101, #indexes 0 thru 100 = 101 elements (these are % numbers)
                'All_Death_Times': [0]*101,

                "Attack": {
                    #make these all linkedlists which apparently is using deque()
                    "kill_info": [[] for _ in range(160)],
                    "death_info": [[] for _ in range(160)],
                    #WE ARE DITCHING THE PLANT INFO!, making a separate dictionary that gets ALL plant data and defuse data(not just specific player)
                },
                "Defense": {
                    "kill_info": [[] for _ in range(160)],
                    "death_info": [[] for _ in range(160)]
                }
            }

    output_dict[map_name]['num_matches'] += 1 #add one match to the num matches analyzed

    if(analyze_all_players):
        
        get_data_for_all_players(curr_match, map_name, output_dict)
    else:
        get_data_for_player(curr_match, map_name, puuid, output_dict)
    
    return output_dict


#Accesses API(indirectly)
#returns specified(puuid) player's data 
#data structure image link: [to be added]
def get_all_location_data(puuid: str, all_match_ids: [], examined_matches: set = None, to_return_dict:dict = {}):
    print("START!")
    for index, match_id in enumerate(all_match_ids):

        if(examined_matches is not None and match_id in examined_matches):
            continue

        curr_match = get_match_details(match_id, api_key)

        print("\nExamining Match [", index+1, "]")

        to_return_dict = analyze_match(curr_match, puuid, to_return_dict)

        if(examined_matches is not None):
            examined_matches.add(match_id)

    return to_return_dict

#Accesses API(indirectly)
#returns an array of all of the match ids that we have access to (from the api)
def get_player_match_IDs(puuid: str):
    
    match_history: dict = get_raw(puuid, api_key, mode ,0 ,25)

    total_elements = match_history['Total']
    
    history = []

    history.extend([match['MatchID'] for match in match_history['History']])

    if total_elements > 25:#matchhistory: limit = 25 at a time
        for i in range(25, total_elements, 25):
            next_batch = get_raw(puuid, api_key, mode, i, i+25)
            history.extend([match['MatchID'] for match in next_batch['History']])

    return history


#TAKES LOT OF TIME !! !
#creates a dictionary where the keys are puuids, and the values are GEA structures corresponding to each puuid
#Only use this one if you need to distinguish between individual players stats
def get_individual_data_for_players(puuids: list):
    toReturn={}
    for currID in puuids:
        
        toReturn[currID] = get_all_location_data(currID,get_player_match_IDs(currID))

    return toReturn



#UNUSED
#adds multiple players ids to one big GEA structure
def get_all_players_location_data(players_match_data: dict, api_key: str):
    """
    players_match_data: dict with player puuid as keys and list of match IDs as values
    {
        "puuid1": [match_id1, match_id2, ...],
        "puuid2": [match_id1, match_id2, ...],
        ...
    }
    """
    to_return_dict = {}
    for puuid, match_ids in players_match_data.items():
        to_return_dict = process_player_match_history(puuid, match_ids, to_return_dict, api_key)
    return to_return_dict
#UNUSED
#helper for above
def process_player_match_history(puuid: str, match_ids: [], to_return_dict: dict, api_key: str):
    for index, match_id in enumerate(match_ids):
        curr_match = get_match_details(match_id, api_key)
        
        print("\nExamining Match [", index+1, "]")

        to_return_dict = analyze_match(curr_match, puuid, to_return_dict)
    return to_return_dict


#combines multiple GEA structures into one
def combine(data_structures):
    combined = {}

    for data in data_structures:
        # print(json.dumps(data, indent=4))
        for map_name, map_data in data.items():
            if map_name not in combined:
                combined[map_name] = {
                    'num_matches': 0
                }
            combined[map_name]['num_matches'] += map_data['num_matches']

            phase = "Attack"
            phase_data = map_data[phase]
            if phase not in combined[map_name]:
                combined[map_name][phase] = {
                    "kill_info": [[] for _ in range(160)],
                    "death_info": [[] for _ in range(160)]
                }
            
            for i in range(160):

                # print(type(combined))
                # print(type(combined[map_name]))
                # print(type(combined[map_name][phase]))
                # print(type(combined[map_name][phase]["kill_info"]))
                # print((combined[map_name][phase]["kill_info"]))

                # print(type(combined[map_name][phase]["kill_info"][i]))
                # print("a")
                # print(type(phase_data))
                # print(type(phase_data["kill_info"][i]))
                # print(type(phase_data["death_info"][i]))
                # print(type())


                combined[map_name][phase]["kill_info"][i].extend(phase_data["kill_info"][i])
                combined[map_name][phase]["death_info"][i].extend(phase_data["death_info"][i])


            phase = "Defense"
            phase_data = map_data[phase]
            if phase not in combined[map_name]:
                combined[map_name][phase] = {
                    "kill_info": [[] for _ in range(160)],
                    "death_info": [[] for _ in range(160)]
                }
            
            for i in range(160):

                # print(type(combined))
                # print(type(combined[map_name]))
                # print(type(combined[map_name][phase]))
                # print(type(combined[map_name][phase]["kill_info"]))
                # print((combined[map_name][phase]["kill_info"]))

                # print(type(combined[map_name][phase]["kill_info"][i]))
                # print("a")
                # print(type(phase_data))
                # print(type(phase_data["kill_info"][i]))
                # print(type(phase_data["death_info"][i]))
                # print(type())


                combined[map_name][phase]["kill_info"][i].extend(phase_data["kill_info"][i])
                combined[map_name][phase]["death_info"][i].extend(phase_data["death_info"][i])

    return combined

In [5]:
#creates GEA structure for one player
#PLAYER SPECIFIC
def analyze_individual(name: str, tag:str):

    puuid = get_puuid(name, tag, api_key)['data']['puuid']
    history = get_player_match_IDs(puuid) 

    print(f'   Number of Matches: [{len(history)}]')

    return get_all_location_data(puuid, history) 


#creates a GEA strucutre for every player in allPlayers
# {
# [puuid] -> GEA structure,
# [puuid] -> GEA structure
#}
#PLAYER SPECIFIC
def analyze_group(allPlayers: list):
    allPuuids = []
    for player in allPlayers:
        allPuuids.append(get_puuid(player['name'], player['tag'], api_key)['data']['puuid'])
    
    return get_individual_data_for_players(allPuuids)


#creates one GEA structure for all mathes played by allPlayers
#NOT PLAYER SPECIFIC
def compile_match_data(allPlayers: list):

    #create a big list of match IDS
    allMatchIDs = []
    for player in allPlayers:
        allMatchIDs.extend(get_player_match_IDs(get_puuid(player['name'], player['tag'], api_key)['data']['puuid']))

    print(f'   Number of Matches: [{len(allMatchIDs)}]')

    #keep track of examined matches to avoid duplicates
    examined_matches = set()
    return get_all_location_data("", allMatchIDs, examined_matches)
    


#designated function for dealing with the fixed GEA data
#inputs the file names
def update_fixed_data(allPlayers: list, fixed_data: str, examined_matches: str):
    #read
    # Load the fixed_data file into a dictionary
    if os.path.exists(fixed_data):
        with open(fixed_data, 'r') as file:
            fixed_data_dict = json.load(file)
    else:
        fixed_data_dict = {}

    # Load the examined_matches file into a set
    if os.path.exists(examined_matches):
        with open(examined_matches, 'r') as file:
            examined_matches_set = set(json.load(file))
    else:
        examined_matches_set = set()
    


    #data processing
    allMatchIDs = []
    for player in allPlayers:
        allMatchIDs.extend(get_player_match_IDs(get_puuid(player['name'], player['tag'], api_key)['data']['puuid']))

    print(f'   Number of Matches: [{len(allMatchIDs)}]')

    #keep track of examined matches to avoid duplicates
    fixed_data_dict = get_all_location_data("", allMatchIDs, examined_matches =examined_matches_set, to_return_dict= fixed_data_dict)
    


    #write
    # Save the updated fixed_data_dict to a temporary file and then rename it
    with tempfile.NamedTemporaryFile('w', delete=False) as temp_file:
        json.dump(fixed_data_dict, temp_file, indent=4)
        temp_file_path = temp_file.name
    os.replace(temp_file_path, fixed_data)

    # Save the updated examined_matches_set to a temporary file and then rename it
    with tempfile.NamedTemporaryFile('w', delete=False) as temp_file:
        json.dump(list(examined_matches_set), temp_file, indent=4)
        temp_file_path = temp_file.name
    os.replace(temp_file_path, examined_matches)


In [7]:
#INPUT HERE
# player_GEA_data = analyze_individual(name = "GEA joli", tag = "brrr")


leaderboard_players = [
    # {
    #     'name': 'hm yes okay',
    #     'tag' : '7236'
    # },
    # {
    #     'name': 'Calamity',
    #     'tag' : '6737'
    # },
    {
        'name': 'ttv asianvyn',
        'tag' : 'LFT'
    },
    {
        'name': 'inspire',
        'tag' : 'Julia'
    }
    
]

leaderboard_GEA_data = compile_match_data(leaderboard_players)

update_fixed_data(leaderboard_players, rf'data\fixed_GEA_data\leaderboard_data\alldata.json',rf'data\fixed_GEA_data\leaderboard_data\examined_matchIDs.json')

Making request...
			request made.
				Alleged requests made: [1]/30
Making request...
			request made.
				Alleged requests made: [2]/30
Making request...
			request made.
				Alleged requests made: [3]/30
Making request...
			request made.
				Alleged requests made: [4]/30
Making request...
			request made.
				Alleged requests made: [5]/30
Making request...
			request made.
				Alleged requests made: [6]/30
Making request...
			request made.
				Alleged requests made: [7]/30
Making request...
			request made.
				Alleged requests made: [8]/30
Making request...
			request made.
				Alleged requests made: [9]/30
   Number of Matches: [140]
START!
Making request...
			request made.
				Alleged requests made: [10]/30

Examining Match [ 1 ]
----CURRENT MAP: [Bind]----
Making request...
			request made.
				Alleged requests made: [11]/30

Examining Match [ 2 ]
----CURRENT MAP: [Ascent]----
Making request...
			request made.
				Alleged requests made: [12]/30

Examining Match [ 3 ]
----CUR

In [ ]:

#ignore:
    #I will change up the get_all_location_data function for more customizability, and create more data structures
    #instead of having return values it will just mutate multiple dictionaries probably

    #   kill + death distance
    #   fix kill and death time!!!! and put in separate map
    #   maybe more cool things

In [5]:
#READ THIS
#DATA PLOTTING FUNCTIONS(feel free to collapse)
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from scipy.ndimage import gaussian_filter
import matplotlib.colors as mcolors
from PIL import Image
from sklearn.neighbors import KernelDensity
import math

# Function to convert game coordinates to image coordinates
#auxiliary function
def game_to_image_coords(game_x, game_y, img_width, img_height, x_multiplier, x_scalar_add, y_multiplier, y_scalar_add):
    x = game_y * x_multiplier + x_scalar_add
    y = game_x * y_multiplier + y_scalar_add
    
    # Scale to image dimensions
    x *= img_width
    y = (1-y) * img_height
    return x, y






#plots kill heatmap, death heatmap, and combines them to make a positioning heatmap
def plot_data(all_data: dict, map_name: str, attack_options: dict = None, defense_options: dict = None, scatterplot: bool = False, logscale: bool = False):
    padding = 200

    if map_name not in all_data:
        print(f"No recorded matches on {map_name}.")
        return
    
    #GETTING THE IMAGE
    map_img = Image.open(rf"map_images\{map_name} Map.png")
    map_img = np.array(map_img)  # Convert the image to a NumPy array
    img_height, img_width, _ = map_img.shape  # Get image dimensions


    

    selected_map = all_data[map_name]


    combined_data = {
            'kill': [],
            'death': []
    }
    
    attack = attack_options is not None 
    defense = defense_options is not None 
    if(not attack and not defense):
       print("please select an option ")
       return

    if(attack):
        
        if(attack_options['kills'] != None):
            if 0 <= attack_options['kills']['lower'] <= 159 and 0 <= attack_options['kills']['upper'] <= 159 and attack_options['kills']['lower'] <= attack_options['kills']['upper']:
                for i in range(attack_options['kills']['lower'], attack_options['kills']['upper'] + 1):
                    curr_kill_value = selected_map['Attack']['kill_info'][i]
                    combined_data['kill'].extend(curr_kill_value)

        if(attack_options['deaths'] != None):
            if 0 <= attack_options['deaths']['lower'] <= 159 and 0 <= attack_options['deaths']['upper'] <= 159 and attack_options['deaths']['lower'] <= attack_options['deaths']['upper']:
                for i in range(attack_options['deaths']['lower'], attack_options['deaths']['upper'] + 1):
                    curr_death_value = selected_map['Attack']['death_info'][i]
                    combined_data['death'].extend(curr_death_value)


    if(defense):
        if(defense_options['kills'] != None):
            if 0 <= defense_options['kills']['lower'] <= 159 and 0 <= defense_options['kills']['upper'] <= 159 and defense_options['kills']['lower'] <= defense_options['kills']['upper']:
                for i in range(defense_options['kills']['lower'], defense_options['kills']['upper'] + 1):
                    curr_kill_value = selected_map['Defense']['kill_info'][i]
                    combined_data['kill'].extend(curr_kill_value)

        if(defense_options['deaths'] != None):
            if 0 <= defense_options['deaths']['lower'] <= 159 and 0 <= defense_options['deaths']['upper'] <= 159 and defense_options['deaths']['lower'] <= defense_options['deaths']['upper']:
                for i in range(defense_options['deaths']['lower'], defense_options['deaths']['upper'] + 1):
                    curr_death_value = selected_map['Defense']['death_info'][i]
                    combined_data['death'].extend(curr_death_value)


    bw = 0.06

    # Plot kill data
    if combined_data['kill']:
        x_coords = [loc['Location']['x'] for loc in combined_data['kill'] ]#x,y swapped!!!!
        y_coords = [loc['Location']['y'] for loc in combined_data['kill'] ]

        # Convert game coordinates to image coordinates
        img_coords = [game_to_image_coords(gx, gy, img_width, img_height, mapCoords[map_name]['xMultiplier'], mapCoords[map_name]['xScalar'], mapCoords[map_name]['yMultiplier'], mapCoords[map_name]['yScalar']) for gx, gy in zip(x_coords, y_coords)]
        img_coords = np.array(img_coords)


        # kde = KernelDensity(bandwidth=bw, kernel='gaussian') #NEW KDE FROM SKLEARN
        # kde.fit(img_coords)
        # xgrid = np.linspace(0, img_width, 200)
        # ygrid = np.linspace(0, img_height, 200)
        # Xgrid, Ygrid = np.meshgrid(xgrid, ygrid)
        # grid_points = np.vstack([Xgrid.ravel(), Ygrid.ravel()]).T
        # Z_Kill = np.exp(kde.score_samples(grid_points)).reshape(Xgrid.shape)

        kde = gaussian_kde(img_coords.T, bw_method=bw)
        xgrid = np.linspace(0, img_width, 200)
        ygrid = np.linspace(0, img_height, 200)
        Xgrid, Ygrid = np.meshgrid(xgrid, ygrid)
        grid_points = np.vstack([Xgrid.ravel(), Ygrid.ravel()]).T
        Z_Kill = kde(grid_points.T).reshape(Xgrid.shape)



        # Create a plot
        fig, ax = plt.subplots(figsize=(8, 8))

        # Display the map layout image with adjustable aspect ratio
        ax.imshow(map_img, extent=(0, img_width, 0, img_height), aspect='auto', alpha=1)

        if(logscale):
            Z_Kill = np.log(Z_Kill+1)  # Adding 1 to avoid log(0)
        # Display the heatmap
        heatmap = ax.imshow( Z_Kill, extent=(0, img_width, 0, img_height), origin='lower', cmap='magma', alpha=0.6, aspect='equal')

        # Plot the points directly on the image
        if(scatterplot):
            ax.scatter(img_coords[:, 0], img_coords[:, 1], color='green', s=25, alpha=0.3)  # Adjust color, size, and alpha as needed

        # Label axes
        ax.set_xlabel('X Coordinate')
        ax.set_ylabel('Y Coordinate')

        # Add a colorbar to serve as a key for the heatmap colors
        cbar = plt.colorbar(heatmap, ax=ax)
        cbar.set_label('Density')
        
        #set title
        if attack and defense:
            plt.title(f'Kill Locations on {map_name} \n On Attack and Defense')
        elif attack:
            plt.title(f'Kill Locations on {map_name} \n On Attack')
        elif defense:
            plt.title(f'Kill Locations on {map_name} \n On Defense')

        # Show the plot
        plt.show()

            
        
            
            # direction = event.get('ViewRadians', 0)
            # dy = np.cos(direction) * 400 #x,y swapped!!!!
            # dx = np.sin(direction) * 400
            # plt.arrow(x, y, dx, dy, color='green', head_width=10, head_length=20, alpha=0.3, linewidth=0.5)
    
    # Plot death data
    if combined_data['death']:
        x_coords = [loc['Location']['x'] for loc in combined_data['death'] ]#x,y swapped!!!!
        y_coords = [loc['Location']['y'] for loc in combined_data['death'] ]
        

        # Convert game coordinates to image coordinates
        img_coords = [game_to_image_coords(gx, gy, img_width, img_height, mapCoords[map_name]['xMultiplier'], mapCoords[map_name]['xScalar'], mapCoords[map_name]['yMultiplier'], mapCoords[map_name]['yScalar']) for gx, gy in zip(x_coords, y_coords)]
        img_coords = np.array(img_coords)


        # kde = KernelDensity(bandwidth=bw, kernel='gaussian') #NEW KDE FROM SKLEARN
        # kde.fit(img_coords)
        # xgrid = np.linspace(0, img_width, 200)
        # ygrid = np.linspace(0, img_height, 200)
        # Xgrid, Ygrid = np.meshgrid(xgrid, ygrid)
        # grid_points = np.vstack([Xgrid.ravel(), Ygrid.ravel()]).T
        # Z_Death = np.exp(kde.score_samples(grid_points)).reshape(Xgrid.shape)
        
        kde = gaussian_kde(img_coords.T, bw_method=bw)
        xgrid = np.linspace(0, img_width, 200)
        ygrid = np.linspace(0, img_height, 200)
        Xgrid, Ygrid = np.meshgrid(xgrid, ygrid)
        grid_points = np.vstack([Xgrid.ravel(), Ygrid.ravel()]).T
        Z_Death = kde(grid_points.T).reshape(Xgrid.shape)



        # Create a plot
        fig, ax = plt.subplots(figsize=(8, 8))

        # Display the map layout image with adjustable aspect ratio
        ax.imshow(map_img, extent=(0, img_width, 0, img_height), aspect='auto', alpha=1)

        if(logscale):
            Z_Death = np.log(Z_Death+1)  
        # Display the heatmap
        heatmap = ax.imshow( Z_Death, extent=(0, img_width, 0, img_height), origin='lower', cmap='magma', alpha=0.6, aspect='equal')

        # Plot the points directly on the image
        if(scatterplot):
            ax.scatter(img_coords[:, 0], img_coords[:, 1], color='red', s=25, alpha=0.3)  # Adjust color, size, and alpha as needed

        # Label axes
        ax.set_xlabel('X Coordinate')
        ax.set_ylabel('Y Coordinate')

        # Add a colorbar to serve as a key for the heatmap colors
        cbar = plt.colorbar(heatmap, ax=ax)
        cbar.set_label('Density')
        
        
        #set title
        if attack and defense:
            plt.title(f'Death Locations on {map_name} \n On Attack and Defense')
        elif attack:
            plt.title(f'Death Locations on {map_name} \n On Attack')
        elif defense:
            plt.title(f'Death Locations on {map_name} \n On Defense')

        # Show the plot
        plt.show()


    # Compute the difference between the two KDEs and apply log scale
    # Compute the difference between the two KDEs
    # Z_diff = Z_Kill - Z_Death 
    # Z_diff = np.where(np.abs(Z_diff) < 0.15, 0, Z_diff)
    # Z_diff = Z_diff/(Z_Kill+Z_Death + 10000)

    # Z_Kill = np.where(Z_Kill < 1, 0, Z_Kill)
    # Z_Death = np.where(Z_Death < 1, 0, Z_Death)
    Z_diff = ((Z_Kill + 1) / (Z_Death + 1))#*(Z_Kill + Z_Death)*(1000)

    Z_diff = gaussian_filter(Z_diff, sigma=4)

    # Create a custom colormap with a nonlinear transition to white
    colors = [(0, 'white'), (0.175, 'red'), (0.49,"black"),(0.51,"black"),(0.815, 'green'), (1, 'white')]
    cmap_diff = mcolors.LinearSegmentedColormap.from_list('custom_bwr', colors)


    # Create a plot
    fig, ax = plt.subplots(figsize=(8, 8))

    # Display the map layout image with adjustable aspect ratio
    ax.imshow(map_img, extent=(0, img_width, 0, img_height), aspect='auto', alpha=0.5)

    # Display the heatmap
    heatmap = ax.imshow( Z_diff, extent=(0, img_width, 0, img_height), origin='lower', cmap=cmap_diff, alpha=0.6, aspect='equal', vmin=Z_diff.min(), vmax=Z_diff.max())
    
    #Display the heatmap ORIGINAL
    # heatmap = ax.imshow( Z_diff, extent=(0, img_width, 0, img_height), origin='lower', cmap=cmap_diff, alpha=0.6, aspect='equal', vmin=-np.abs(Z_diff).max(), vmax=np.abs(Z_diff).max())

#SAVED THIS FOR SETTINGS
    # plt.imshow(Z_diff_log, cmap=cmap_diff, origin='lower', extent=[x_min, x_max, y_min, y_max], vmin=-np.abs(Z_diff_log).max(), vmax=np.abs(Z_diff_log).max(), alpha = 0.65)
    
    # Add a colorbar to serve as a key for the heatmap colors
    cbar = plt.colorbar(heatmap, ax=ax)
    cbar.set_label('<--Bad     Good-->')
    
    #set title
    if attack and defense:
        plt.title(f'Positioning Guide on {map_name} \n On Attack and Defense')
    elif attack:
        plt.title(f'Positioning Guide on {map_name} \n On Attack')
    elif defense:
        plt.title(f'Positioning Guide on {map_name} \n On Defense')


    plt.gca().set_aspect('equal', adjustable='box')
    plt.show()


    # plt.title(f"Game Events Visualization on {map_name}")
    # plt.xlabel('Y Coordinate')#x,y swapped!!!!
    # plt.ylabel('X Coordinate')
    # plt.grid(True)
    
    # # Add a legend to explain color codes
    # plt.scatter([], [], color='green', label='Kill Locations', alpha=0.3, s=50)
    # plt.scatter([], [], color='red', label='Death Locations', alpha=0.3, s=50)
    # plt.legend(loc='upper right')

    # plt.show()


#combines the kill and death data to generate a general 'activity' heatmap
def plot_activity(all_data: dict, map_name: str, attack_options: dict = None, defense_options: dict = None, scatterplot: bool = False, logscale: bool = False):
    
    if map_name not in all_data:
        print(f"No recorded matches on {map_name}.")
        return
    
    #GETTING THE IMAGE
    map_img = Image.open(rf"map_images\{map_name} Map.png")
    map_img = np.array(map_img)  # Convert the image to a NumPy array
    img_height, img_width, _ = map_img.shape  # Get image dimensions


 
    selected_map = all_data[map_name]


    combined_data= []
    attack = attack_options is not None  #check which options we want
    defense = defense_options is not None 
    if(not attack and not defense):
       print("please select an option ")
       return

    if(attack):
        if(attack_options['kills'] != None):
            if 0 <= attack_options['kills']['lower'] <= 159 and 0 <= attack_options['kills']['upper'] <= 159 and attack_options['kills']['lower'] <= attack_options['kills']['upper']:
                for i in range(attack_options['kills']['lower'], attack_options['kills']['upper'] + 1):
                    curr_kill_value = selected_map['Attack']['kill_info'][i]
                    combined_data.extend(curr_kill_value)

        if(attack_options['deaths'] != None):
            if 0 <= attack_options['deaths']['lower'] <= 159 and 0 <= attack_options['deaths']['upper'] <= 159 and attack_options['deaths']['lower'] <= attack_options['deaths']['upper']:
                for i in range(attack_options['deaths']['lower'], attack_options['deaths']['upper'] + 1):
                    curr_death_value = selected_map['Attack']['death_info'][i]
                    combined_data.extend(curr_death_value)


    if(defense):
        if(defense_options['kills'] != None):
            if 0 <= defense_options['kills']['lower'] <= 159 and 0 <= defense_options['kills']['upper'] <= 159 and defense_options['kills']['lower'] <= defense_options['kills']['upper']:
                for i in range(defense_options['kills']['lower'], defense_options['kills']['upper'] + 1):
                    curr_kill_value = selected_map['Defense']['kill_info'][i]
                    combined_data.extend(curr_kill_value)

        if(defense_options['deaths'] != None):
            if 0 <= defense_options['deaths']['lower'] <= 159 and 0 <= defense_options['deaths']['upper'] <= 159 and defense_options['deaths']['lower'] <= defense_options['deaths']['upper']:
                for i in range(defense_options['deaths']['lower'], defense_options['deaths']['upper'] + 1):
                    curr_death_value = selected_map['Defense']['death_info'][i]
                    combined_data.extend(curr_death_value)


    
    x_coords = [loc['Location']['x'] for loc in combined_data ]
    y_coords = [loc['Location']['y'] for loc in combined_data ]

    # Convert game coordinates to image coordinates
    img_coords = [game_to_image_coords(gx, gy, img_width, img_height, mapCoords[map_name]['xMultiplier'], mapCoords[map_name]['xScalar'], mapCoords[map_name]['yMultiplier'], mapCoords[map_name]['yScalar']) for gx, gy in zip(x_coords, y_coords)]
    img_coords = np.array(img_coords)


    kde = KernelDensity(bandwidth=12, kernel='gaussian') #NEW KDE FROM SKLEARN
    kde.fit(img_coords)
    xgrid = np.linspace(0, img_width, 200)
    ygrid = np.linspace(0, img_height, 200)
    Xgrid, Ygrid = np.meshgrid(xgrid, ygrid)
    grid_points = np.vstack([Xgrid.ravel(), Ygrid.ravel()]).T
    Z = np.exp(kde.score_samples(grid_points)).reshape(Xgrid.shape)

    # Create a plot
    fig, ax = plt.subplots(figsize=(8, 8))

    # Display the map layout image with adjustable aspect ratio
    ax.imshow(map_img, extent=(0, img_width, 0, img_height), aspect='auto', alpha=1)

    if(logscale):
        Z = np.log(Z+ 0.000004)  # Adding 1 to avoid log(0)
    # Display the heatmap
    heatmap = ax.imshow( Z, extent=(0, img_width, 0, img_height), origin='lower', cmap='magma', alpha=0.6, aspect='equal')

    # Plot the points directly on the image
    if(scatterplot):
        ax.scatter(img_coords[:, 0], img_coords[:, 1], color='blue', s=10, alpha=0.4)  # Adjust color, size, and alpha as needed

    # Label axes
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')

    # Add a colorbar to serve as a key for the heatmap colors
    cbar = plt.colorbar(heatmap, ax=ax)
    cbar.set_label('Density')
    
    if attack and defense:
        plt.title(f'Kills & Death Locations on {map_name} \n On Attack and Defense')
    elif attack:
        plt.title(f'Kills & Death Locations on {map_name} \n On Attack')
    elif defense:
        plt.title(f'Kills & Death Locations on {map_name} \n On Defense')

    # Show the plot
    plt.show()






    # Plot the scatter plot of points
    
    # plt.figure(figsize=(18, 8))
    # if(True):
    #     plt.subplot(1, 2, 1)
    #     plt.scatter(x_coords, y_coords, c='blue', alpha=0.3, s=30)  # Reduced size to half
        
        


    #     plt.title('All Locations')
    #     plt.xlabel('X coordinate')
    #     plt.ylabel('Y coordinate')
    #     plt.imshow(map_img, extent=[x_min-700, x_max+900, y_min-1500, y_max+1950], aspect='auto', alpha=1)

    #     # Plot the KDE contour map
    #     plt.subplot(1, 2, 2)
    #     contour = plt.contourf(X, Y, Z_Kill, levels=10, cmap='magma')  # Adjust levels as needed
    #     plt.colorbar(contour)
    #     plt.title('Topographical Heatmap with KDE')
    #     plt.xlabel('X coordinate')
    #     plt.ylabel('Y coordinate')
    #     plt.show()



    # # Apply logarithmic scaling to enhance contrast
    # Z1_log = np.log(Z_Kill+ 0.00000004)  # Adding 1 to avoid log(0)
    # #plt.contourf(X, Y, Z1_log, levels=10, cmap='magma')  # Adjust levels as needed
    
    # plt.imshow(map_img, extent=[x_min-700, x_max+900, y_min-1500, y_max+1950], aspect='auto', alpha=1)

    # plt.imshow(Z1_log, cmap='magma', origin='lower', extent=[x_min, x_max, y_min, y_max], alpha = 0.65)


    # #plt.imshow(Z1_log, cmap='magma', origin='lower', extent=[x_min+ 3*padding, x_max+ 3*padding, y_min+ 3*padding, y_max+ 3*padding])
    # plt.colorbar(label='Log Density')
    # plt.title('Logarithmic Scale Heatmap of KDE(Kills + Deaths)')
    # plt.xlabel('X coordinate')
    # plt.ylabel('Y coordinate')

    # plt.tight_layout()
    # plt.gca().set_aspect('equal', adjustable='box')
    # plt.show()


#plots all data for all maps in a standard way(coordinate points)
def plot_all_data(all_data):
    combined_data_by_map = {}

    # Combine all data into one dictionary
    for map_name, sides in all_data.items():

        
        
        combined_data_by_map[map_name] = {
            'kill': [kill for side in sides.values() for kills in side.get('kill_info', []) for kill in kills ],
            'death': [death for side in sides.values() for deaths in side.get('death_info', []) for death in deaths ]
        }

    # Plotting the data
    for map_name, data in combined_data_by_map.items():
        plt.figure(figsize=(10, 8))

        

        
        # Plot kill data
        for event in data['kill']:
            if 'Location' in event and event['Location']:
                y = event['Location']['x'] 
                x = event['Location']['y']
                plt.scatter(x, y, marker='o', color='green', alpha=0.3, s=15)  # Reduced size to half
                direction = event.get('ViewRadians', 0)
                dx = np.cos(direction) * 400
                dy = np.sin(direction) * 400
                plt.arrow(x, y, dx, dy, color='green', head_width=10, head_length=20, alpha=0.3, linewidth=0.5)
        
        # Plot death data
        if data['death']:
            y_coords = [loc['Location']['x'] for loc in data['death'] ]
            x_coords = [loc['Location']['y'] for loc in data['death'] ]
            plt.scatter(x_coords, y_coords, c='red', alpha=0.3, s=15)  # Reduced size to half

        plt.title(f"Game Events Visualization on {map_name}")
        plt.xlabel('Y Coordinate')
        plt.ylabel('X Coordinate')
        plt.grid(True)
        
        # Add a legend to explain color codes
        plt.scatter([], [], color='green', label='Kill Locations', alpha=0.3, s=50)
        plt.scatter([], [], color='red', label='Death Locations', alpha=0.3, s=50)
        plt.legend(loc='upper right')

        plt.show()


def plot_buy_frequency(all_data: dict, map_name: str):
    if map_name not in all_data:
        print(f"No recorded matches on {map_name}.")
        return
    
    selected_map = all_data[map_name]
    attack_buy_freq = [0] * 160
    defense_buy_freq = [0] * 160

    for i, (kill_info, death_info) in enumerate(zip(selected_map['Attack']['kill_info'], selected_map['Attack']['death_info'])):
        attack_buy_freq[i] += len(kill_info) + len(death_info)

    for i, (kill_info, death_info) in enumerate(zip(selected_map['Defense']['kill_info'], selected_map['Defense']['death_info'])):
        defense_buy_freq[i] += len(kill_info) + len(death_info)
    plt.figure(figsize=(10, 8))
    plt.subplot(2, 1, 1)
    plt.bar([i * 50 for i in range(len(attack_buy_freq))], attack_buy_freq, alpha=0.5, color='red', label='Attack')
    plt.title(f'Attack Buy Frequency on {map_name}')
    plt.xlabel('Index')
    plt.ylabel('Frequency')
    plt.legend()

    plt.subplot(2, 1, 2)
    plt.bar([i * 50 for i in range(len(defense_buy_freq))], defense_buy_freq, alpha=0.5, label='Defense')
    plt.title(f'Defense Buy Frequency on {map_name}')
    plt.xlabel('Index')
    plt.ylabel('Frequency')
    plt.legend()

    plt.tight_layout()
    plt.show()


In [6]:
#INPUT HERE

def buy_values(buy_name: str):
    if(buy_name == 'pistol'):
        return 0,1000
    elif(buy_name == 'rifle'):
        return 3900, 5000
    elif(buy_name == 'all'):
        return 0, 7950
    elif(buy_name == 'fullbuy'):
        return 3600, 7950
    elif(buy_name == 'midbuy'):
        return 1000, 3900
    elif(buy_name == 'eco'):
        return 0, 3000
    

a_lower, a_upper = buy_values('pistol') #ATTACKERS BUY

d_lower, d_upper = buy_values('pistol') #DEFENDERS BUY


attack_options = {
    "kills": {"lower": a_lower//50, "upper": a_upper//50}, #between 0 160 
    "deaths": {"lower": a_lower//50, "upper": a_upper//50}

}
defense_options = {
    "kills": {"lower": d_lower//50, "upper": d_upper//50}, #between 0 160
    "deaths": {"lower": d_lower//50, "upper":d_upper//50}
}



with open(r'data\fixed_GEA_data\leaderboard_data\alldata.json', 'r') as file:
    data = json.load(file)

print(data["Sunset"]["num_matches"])
# plot_buy_frequency(data,'Ascent')
plot_activity(data, 'Sunset',defense_options = defense_options, logscale = False)
plot_data(data, 'Sunset', attack_options = attack_options, logscale = True, scatterplot = False) #the problem is: that when i take logscale the value is negative, which I dont want

# print(test_allData["Ascent"]["num_matches"])
# plot_activity(test_allData, 'Ascent', attack_options = attack_options, defense_options = defense_options, logscale = True)
# plot_data(test_allData, 'Ascent', attack_options = attack_options,logscale = True, scatterplot = False)



35


NameError: name 'mapCoords' is not defined

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
from PIL import Image

# Function to convert game coordinates to image coordinates
def game_to_image_coords(game_x, game_y, img_width, img_height, x_multiplier, x_scalar_add, y_multiplier, y_scalar_add):
    x = game_y * x_multiplier + x_scalar_add
    y = game_x * y_multiplier + y_scalar_add
    
    # Scale to image dimensions
    x *= img_width
    y *= img_height
    
    return x, y

# Generate some sample data
np.random.seed(0)
game_x = np.random.normal(0, 1000, 100)
game_y = np.random.normal(0, 1000, 100)

# Load the map layout image
map_img = Image.open(rf'map_images\Lotus Map.png')
map_img = np.array(map_img)  # Convert the image to a NumPy array
img_height, img_width, _ = map_img.shape  # Get image dimensions

# Define the constants for conversion
x_multiplier = 0.000072  # Replace with actual multiplier
x_scalar_add = 0.454789  # Replace with actual scalar addition
y_multiplier =-0.000072  # Replace with actual multiplier
y_scalar_add =0.917752  # Replace with actual scalar addition

# Convert game coordinates to image coordinates
img_coords = [game_to_image_coords(gx, gy, img_width, img_height, x_multiplier, x_scalar_add, y_multiplier, y_scalar_add) for gx, gy in zip(game_x, game_y)]
img_coords = np.array(img_coords)

# Kernel Density Estimation
kde = KernelDensity(bandwidth=20, kernel='gaussian')
kde.fit(img_coords)
xgrid = np.linspace(0, img_width, 100)
ygrid = np.linspace(0, img_height, 100)
Xgrid, Ygrid = np.meshgrid(xgrid, ygrid)
grid_points = np.vstack([Xgrid.ravel(), Ygrid.ravel()]).T
Z = np.exp(kde.score_samples(grid_points)).reshape(Xgrid.shape)

# Create a plot
fig, ax = plt.subplots(figsize=(8, 8))

# Display the map layout image with adjustable aspect ratio
ax.imshow(map_img, extent=(0, img_width, 0, img_height), aspect='auto', alpha=0.5)

# Display the heatmap
heatmap = ax.imshow(Z, extent=(0, img_width, 0, img_height), origin='lower', cmap='viridis', alpha=0.6, aspect='equal')

# Plot the points directly on the image
ax.scatter(img_coords[:, 0], img_coords[:, 1], color='red', s=10, alpha=0.8)  # Adjust color, size, and alpha as needed

# Label axes
ax.set_xlabel('X Coordinate')
ax.set_ylabel('Y Coordinate')

# Add a colorbar to serve as a key for the heatmap colors
cbar = plt.colorbar(heatmap, ax=ax)
cbar.set_label('Density')

# Show the plot
plt.show()
